In [2]:
import numpy as np
import pandas as pd
import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package wordnet to C:\Users\Mr.
[nltk_data]     Luke\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
df = pd.read_csv('randomPosts.csv', encoding='utf-8')
df = df[df['Body'].notna()]

df = df[df['PostTypeId'] == 1]

df.head()

print(df)

dict = df.to_dict('index')


# for key in dict:
    # dict[key]['words_tokenized'] = word_tokenize(dict[key]['Body'])

    

             Id  AcceptedAnswerId  AnswerCount  \
1         25277             25456            3   
7         44391             65953           12   
20        62999                 0            9   
47       138261            138792            9   
51       139046            139500            7   
...         ...               ...          ...   
53339  67746074          67754768            1   
53340  67746075                 0            0   
53341  67750181                 0            0   
53347  67754483                 0            1   
53348  67754484                 0            0   

                                                    Body  \
1      <p>There is a <a href="http://stackoverflow.us...   
7      <p>This is related to <a href="https://stackov...   
20     <p>I’ve been trying to install Ms SQL Server 2...   
47     <p>I am having a table Table1 with columns id1...   
51     <p>I need to write code that picks up PGP-encr...   
...                                    

In [4]:
def prepare_text_for_lda(text):
    tokens = word_tokenize(text)
    # tokens = [token for token in tokens if len(token) > 4]
    # tokens = [token for token in tokens if token not in en_stop]
    # tokens = [get_lemma(token) for token in tokens]
    return tokens

In [5]:
def removeFirstLastThree(text):
    text = text[3:]
    # text = text[:3]
    text = text[:len(text)-3]
    return text

In [6]:
import string
def toLowerCase(text):
    text = [word.lower() for word in text]
    return text


In [7]:
from nltk.corpus import stopwords


def removeStopWords(text):
    stop_words=set(stopwords.words("english"))
    filtered_sent=[]
    for w in text:
        if w not in stop_words:
            filtered_sent.append(w)
    return filtered_sent


In [8]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

def applyPStemmer(text):
    ps = PorterStemmer()

    stemmed_words=[]
    for w in text:
        stemmed_words.append(ps.stem(w))

    return stemmed_words


In [9]:
from nltk.tokenize import RegexpTokenizer

def applyPStemmer(text):
    tokenizer = RegexpTokenizer(r"\w+")
    text = tokenizer.tokenize(' '.join(text))
    return text

In [10]:

# nltk.download('wordnet')
from nltk.corpus import wordnet as wn

def get_lemma(text):
    words = []
    for word in text:
        lemma = wn.morphy(word)
        if (len(word) <= 2 or len(word) >= 15 or word == 'code' or word.isnumeric() or word == 'gt' or word == 'lt' or word =='quot' or word == 'pre' or word == 'amp'):
            continue 
        elif lemma is None or word == lemma:
            words.append(word)
        else:
            words.append(lemma)
    return words
from nltk.stem.wordnet import WordNetLemmatizer

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [11]:
import random
text_tokens = []
i = 0
for key in dict:
    tokens = prepare_text_for_lda(dict[key]['Body'])
    tokens = removeFirstLastThree(tokens)
    tokens = toLowerCase(tokens)
    tokens = removeStopWords(tokens)
    tokens = applyPStemmer(tokens)
    tokens = get_lemma(tokens)
    # tokens = removeSymbols(tokens)
    

    # if i < 50:
        # print(tokens)
    text_tokens.append(tokens)
    i += 1

In [12]:
import gensim
from gensim import corpora
dict_STF = corpora.Dictionary(text_tokens)

dictionary = corpora.Dictionary(text_tokens) 
corpus = [dictionary.doc2bow(text) for text in text_tokens]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

print(dict_STF)

Dictionary(118613 unique tokens: ['28sql', 'aspx', 'blockquote', 'com', 'current']...)


In [13]:

NUM_TOPICS = 25
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=2)


In [14]:
for topic in topics:
    print(topic)

(6, '0.085*"name" + 0.056*"value"')
(18, '0.022*"player" + 0.018*"device"')
(10, '0.123*"model" + 0.076*"true"')
(3, '0.071*"version" + 0.048*"plugin"')
(23, '0.061*"step" + 0.055*"float"')
(21, '0.109*"http" + 0.077*"com"')
(8, '0.022*"strong" + 0.019*"like"')
(1, '0.055*"include" + 0.047*"int"')
(16, '0.133*"div" + 0.132*"class"')
(9, '0.056*"int" + 0.050*"array"')
(19, '0.060*"map" + 0.050*"product"')
(13, '0.079*"image" + 0.069*"stack"')
(7, '0.049*"width" + 0.048*"color"')
(15, '0.128*"android" + 0.044*"app"')
(2, '0.147*"java" + 0.093*"org"')
(11, '0.053*"var" + 0.047*"script"')
(5, '0.044*"self" + 0.027*"request"')
(17, '0.042*"print" + 0.031*"line"')
(4, '0.248*"user" + 0.057*"password"')
(14, '0.059*"table" + 0.053*"row"')


In [15]:
pd.options.display.max_colwidth = 500
new_doc = df.iloc[[4]]
print(new_doc)

text = 'when i click on a video, it is redirecting back to homepage, youtube clone'
print(tokens)

# tokens = prepare_text_for_lda(dict[key]['Body'])
tokens = prepare_text_for_lda(text)
tokens = removeFirstLastThree(tokens)
tokens = toLowerCase(tokens)
tokens = removeStopWords(tokens)
tokens = applyPStemmer(tokens)
tokens = get_lemma(tokens)

print(tokens)


new_doc_bow = dictionary.doc2bow(tokens)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))
# new_doc_bow = dictionary.doc2bow(new_doc)
# print(new_doc_bow)
# print(ldamodel.get_document_topics(new_doc_bow))

        Id  AcceptedAnswerId  AnswerCount  \
51  139046            139500            7   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   Body  \
51  <p>I need to write code that picks up PGP-encrypted files from an FTP location and processes them. The files will be encrypted with my public key (not that I have one yet). Obviously, I need a PGP library that I can use from within Microsoft Access. Can you recommend one that is easy to use? </p>\n\n<p>I'm looking for something that doesn't require a huge amount of PKI knowledge. Ideally, someth